In [507]:
import numpy as np
import pandas as pd

In [508]:
train_file=r'C:\Users\VDS\Desktop\Prerna Data Science\pythondata\Data (1)\Data/Titanictrain.csv'

In [509]:
test_file=r'C:\Users\VDS\Desktop\Prerna Data Science\pythondata\Data (1)\Data/Titanictest.csv'

In [510]:
tt_train=pd.read_csv(train_file)
tt_test=pd.read_csv(test_file)
tt_train['data']='train'
tt_test['data']='test'
tt_test['Survived']=np.nan

In [511]:
tt_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,data
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


In [512]:
tt_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,data,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,test,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,test,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,test,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,test,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,test,NaN


In [513]:
tt_test=tt_test[tt_train.columns]

In [514]:
tt_all=pd.concat([tt_train,tt_test],axis=0)

In [515]:
list(zip(tt_all.columns,tt_all.dtypes,tt_all.nunique())) 

[('PassengerId', dtype('int64'), 1309),
 ('Survived', dtype('float64'), 2),
 ('Pclass', dtype('int64'), 3),
 ('Name', dtype('O'), 1307),
 ('Sex', dtype('O'), 2),
 ('Age', dtype('float64'), 98),
 ('SibSp', dtype('int64'), 7),
 ('Parch', dtype('int64'), 8),
 ('Ticket', dtype('O'), 929),
 ('Fare', dtype('float64'), 281),
 ('Cabin', dtype('O'), 186),
 ('Embarked', dtype('O'), 3),
 ('data', dtype('O'), 2)]

In [516]:
tt_all.drop(['PassengerId','Fare','Cabin'],axis=1,inplace=True)

In [517]:
tt_all.dtypes

Survived    float64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Embarked     object
data         object
dtype: object

In [518]:
tt_all.drop(['Name','Ticket'],axis=1,inplace=True)

In [519]:
cat_vars=tt_all.select_dtypes(['object']).columns
cat_vars

Index(['Sex', 'Embarked', 'data'], dtype='object')

In [520]:
for col in cat_vars[:-1]:
    dummy=pd.get_dummies(tt_all[col],drop_first=True,prefix=col) # dropfirtst is removing 1st value, prefix=col is the colname
    tt_all=pd.concat([tt_all,dummy],axis=1)
    del tt_all[col]
    print(col)
del dummy

Sex
Embarked


In [521]:
tt_all.isnull().sum()


Survived      418
Pclass          0
Age           263
SibSp           0
Parch           0
data            0
Sex_male        0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [522]:
tt_all.loc[tt_all['Age'].isnull(),'Age']=tt_all.loc[tt_all['data']=='train','Age'].mean()


In [523]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [524]:
params={'class_weight':['balanced',None], #will give equal weighatage to output variables  for balanced and n
        'penalty':['l1','l2'], #will check l1 is lasso and l2 is ridge
        'C':np.linspace(0.01,1000,10)} #C is inverse of lambda # so it will check 40 values 2*2*10 i.e balanced, none for 1st 2 and 
#then 2nd 2 value for lasso and ridge and 10 valuesbetween 0/001 ad 1000

In [525]:
model=LogisticRegression(fit_intercept=True)

In [526]:
grid_search=GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc",verbose=4) #verbose is used to print back end proceeses

In [528]:
train1=tt_all[tt_all['data']=='train']
del train1['data']
test1=tt_all[tt_all['data']=='test']
test1.drop(['data'],axis=1,inplace=True)

x_train=train1.drop('Survived',axis=1)
y_train=train1['Survived']
x_test=test1.drop('Survived',axis=1)
y_test=test1['Survived']

C:\Users\VDS\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [529]:
train1.dtypes

Survived      float64
Pclass          int64
Age           float64
SibSp           int64
Parch           int64
Sex_male        uint8
Embarked_Q      uint8
Embarked_S      uint8
dtype: object

In [530]:
grid_search.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=0.5873517786561264, total=   1.2s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=0.5433465085638999, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=0.4907085561497327, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=0.516042780748663, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=0.5169320021586616, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l2 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l2, score=0.73972332015

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s


[CV]  C=0.01, class_weight=None, penalty=l1, score=0.6202898550724637, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV]  C=0.01, class_weight=None, penalty=l1, score=0.6617918313570487, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV]  C=0.01, class_weight=None, penalty=l1, score=0.5776069518716578, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV]  C=0.01, class_weight=None, penalty=l1, score=0.568716577540107, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV]  C=0.01, class_weight=None, penalty=l1, score=0.576160280626012, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l2 ...........................
[CV]  C=0.01, class_weight=None, penalty=l2, score=0.7654150197628459, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l2 ...........................
[CV]  C=0.01, class_weight=None, penalty=l2, score=0.7926877470355

[CV]  C=333.34, class_weight=balanced, penalty=l1, score=0.8330213903743315, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l1 .....................
[CV]  C=333.34, class_weight=balanced, penalty=l1, score=0.8720992984349704, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l2 .....................
[CV]  C=333.34, class_weight=balanced, penalty=l2, score=0.8482872200263505, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l2 .....................
[CV]  C=333.34, class_weight=balanced, penalty=l2, score=0.8327404479578393, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l2 .....................
[CV]  C=333.34, class_weight=balanced, penalty=l2, score=0.8566844919786096, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l2 .....................
[CV]  C=333.34, class_weight=balanced, penalty=l2, score=0.8334224598930481, total=   0.0s
[CV] C=333.34, class_weight=balanced, penalty=l2 .....................
[CV]  C=333.34, class_weight

[CV]  C=555.56, class_weight=None, penalty=l2, score=0.8485507246376811, total=   0.0s
[CV] C=555.56, class_weight=None, penalty=l2 .........................
[CV]  C=555.56, class_weight=None, penalty=l2, score=0.8327404479578392, total=   0.0s
[CV] C=555.56, class_weight=None, penalty=l2 .........................
[CV]  C=555.56, class_weight=None, penalty=l2, score=0.8584224598930481, total=   0.0s
[CV] C=555.56, class_weight=None, penalty=l2 .........................
[CV]  C=555.56, class_weight=None, penalty=l2, score=0.8359625668449198, total=   0.0s
[CV] C=555.56, class_weight=None, penalty=l2 .........................
[CV]  C=555.56, class_weight=None, penalty=l2, score=0.8718294657312465, total=   0.0s
[CV] C=666.67, class_weight=balanced, penalty=l1 .....................
[CV]  C=666.67, class_weight=balanced, penalty=l1, score=0.8482872200263505, total=   0.0s
[CV] C=666.67, class_weight=balanced, penalty=l1 .....................
[CV]  C=666.67, class_weight=balanced, penalty=l

[CV]  C=888.89, class_weight=None, penalty=l1, score=0.8485507246376811, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l1 .........................
[CV]  C=888.89, class_weight=None, penalty=l1, score=0.8327404479578392, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l1 .........................
[CV]  C=888.89, class_weight=None, penalty=l1, score=0.8584224598930481, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l1 .........................
[CV]  C=888.89, class_weight=None, penalty=l1, score=0.8359625668449198, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l1 .........................
[CV]  C=888.89, class_weight=None, penalty=l1, score=0.8718294657312465, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l2 .........................
[CV]  C=888.89, class_weight=None, penalty=l2, score=0.8485507246376811, total=   0.0s
[CV] C=888.89, class_weight=None, penalty=l2 .........................
[CV]  C=888.89, class_weight=None, penalty=l2, score

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    3.6s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'class_weight': ['balanced', None], 'penalty': ['l1', 'l2'], 'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=4)

In [531]:
grid_search.best_estimator_

LogisticRegression(C=666.67, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [532]:
logr=grid_search.best_estimator_


In [534]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [535]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 666.67, 'class_weight': None, 'penalty': 'l1'}

Model with rank: 1
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 777.78, 'class_weight': None, 'penalty': 'l1'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 333.34, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 444.45, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 555.56, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 666.67, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 777.78, 'class_weight': None, 'penalty': 'l2'}

Model with rank: 3
Mean validation score: 0.849 (std: 0.014)
Parameters: {'C': 888.89, 'class_weight': None, 'p

In [536]:
logr.fit(x_train,y_train)

LogisticRegression(C=666.67, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [537]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [538]:
train_score=logr.predict_proba(x_train)[:,1]

real=y_train

In [539]:
train_score>0.2

array([False,  True,  True,  True, False, False,  True, False,  True,
        True,  True,  True, False, False,  True,  True, False,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True, False,  True,  True,
       False, False,  True, False,  True, False, False,  True,  True,
        True,  True,  True, False,  True, False, False,  True,  True,
       False,  True, False,  True, False,  True, False,  True,  True,
        True, False, False, False, False, False,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True, False,
       False, False,  True, False, False, False,  True,  True,  True,
       False,  True, False,  True, False, False, False,  True, False,
       False,  True,  True,  True, False,  True,  True, False, False,
       False,  True,  True, False, False,  True,  True,  True, False,
       False, False,

In [540]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
      
    KS=(TP/P)-(FP/N)
    
    
    KS_all.append(KS)

# try out what cutoffs you get when you use F_beta scores with different values of betas [0.5 , 5]
# beta < 1 : you will get cutoff , which is high ( favours precision)
# beta > 1 : you will get cutoff , which is low (favours precision )

In [541]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

0.35000000000000003

In [542]:
logr.intercept_

array([5.49458454])

In [543]:
list(zip(x_train.columns,logr.coef_[0]))

[('Pclass', -1.1566977819716653),
 ('Age', -0.039853814415095605),
 ('SibSp', -0.31297032603643493),
 ('Parch', -0.07296677185307819),
 ('Sex_male', -2.7246912052593553),
 ('Embarked_Q', -0.06083473933361837),
 ('Embarked_S', -0.4635704001912326)]